Importamos las librerías que vamos a utilizar

In [1]:
# Algunas de las de siempre
import numpy as np
import pandas as pd

# Las que vamos a necesitar para hacer los llamados y para lidiar con los datos en formato json
import requests
from flask import  Flask, request, jsonify, render_template
import json
import pygame

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
def play_music(music_file):
    clock = pygame.time.Clock()
    try:
        pygame.mixer.music.load(music_file)
        print ("Music file %s loaded!" % music_file)
    except pygame.error:
        print ("File %s not found! (%s)" % music_file, pygame.get_error())
        return
    pygame.mixer.music.play()
    while pygame.mixer.music.get_busy():
        clock.tick(30)

In [3]:
def play_song(music_file):
    fout = open(music_file,"rb")
    #fout.write(fish)
    fout.close()
    freq = 44100    # audio CD quality
    bitsize = -16   # unsigned 16 bit
    channels = 2    # 1 is mono, 2 is stereo
    buffer = 1024    # number of samples
    pygame.mixer.init(freq, bitsize, channels, buffer)
    # optional volume 0 to 1.0
    pygame.mixer.music.set_volume(0.8)
    try:
        # use the midi file you just saved
        play_music(music_file)
    except KeyboardInterrupt:
        # if user hits Ctrl/C then exit
        # (works only in console mode)
        pygame.mixer.music.fadeout(1000)
        pygame.mixer.music.stop()
        raise SystemExit

In [4]:
def get_song(song_name,df_files):
    archivo = ""
    mask_file = df_files.file_name == song_name + '.mid'
    if (df_files[mask_file].shape[0] > 0):
        archivo = df_files[mask_file].iloc[0]
    else:
        archivo = df_files[df_files['file'].str.contains(song_name)].iloc[0]
    return archivo[0]

In [5]:
import os
classical = 'MIDI folders\midi_selection'

# the dictionary to pass to pandas dataframe
dict = {}
count_files = 0
      
for root, dirs, files in os.walk(classical, topdown=False):
    for name_file in files:
        dict[count_files] = {"file": os.path.join(root, name_file), "file_name": name_file}
        count_files = count_files + 1

df_files = pd.DataFrame.from_dict(dict, "index")
df_files.shape

(5465, 2)

In [6]:
search=False
duracion_notas = 1
amplitud_tonal = 3
ritmica_instrument = 15
ritmica_drums = 20
armonia = 4
dinamica = 4
instrumentacion = 20
tempo = 10
notas_simultaneas = 2
duracion_tema = 15
cant_resultados = 5
url = 'http://localhost:5000/related_songs'

In [25]:
# Hacemos el pedido y vemos los resultados.
# Si todo salió bien en la carpeta "modelos" se tiene que haber generado un .pkl con el modelo entrenado
relateds = requests.get(url, 
                    params = {'search':search,
                             'cant_resultados':cant_resultados ,
                             'instrumentacion':instrumentacion,
                             'ritmica_drums':ritmica_drums ,
                             'ritmica_instrument':ritmica_instrument ,
                             'amplitud_tonal':amplitud_tonal ,
                             'dinamica':dinamica, 
                             'duracion_notas':duracion_notas, 
                             'notas_simultaneas':notas_simultaneas ,
                             'tempo':tempo, 
                             'duracion_tema':duracion_tema ,
                             'armonia':armonia})

df_relateds = pd.DataFrame.from_dict(relateds.json(), "index")
df_relateds

,0
How_Do_You_Do_It,0.627864
Its_Not_Unusual,0.624163
Georgie_Girl,0.599594
Revolution,0.500306
Revolution1,0.486432


In [28]:
recomendado_n = 3

song = df_relateds.index[recomendado_n - 1]
song

play_song(get_song(song,df_files))

Music file MIDI folders\midi_selection\ClassicRock\Seekers\Georgie_Girl.mid loaded!


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "<ipython-input-3-b907ab2e9769>", line 14, in play_song
    play_music(music_file)
  File "<ipython-input-2-5b0900a3a268>", line 11, in play_music
    clock.tick(30)
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Patricio\anaconda3\envs\dhdsblend\lib\site-packages\IPython\core\interactiveshell.py", line 3417, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-28-a6292ee6bc48>", line 6, in <module>
    play_song(get_song(song,df_files))
  File "<ipython-input-3-b907ab2e9769>", line 20, in play_song
    raise SystemExit
SystemExit

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Patricio\anaconda3\envs\dhdsblend\lib\site-packages\IPython\core\ultratb.py", line 1169, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, 

TypeError: object of type 'NoneType' has no len()